In [ ]:
#from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, LongType

# Initialize Spark session
#spark = SparkSession.builder \
    #.appName("Data Cleaning") \
    #.getOrCreate()

# Define schema
schema = StructType([
    StructField("recommendationid", LongType(), True),
    StructField("appid", IntegerType(), True),
    StructField("game", StringType(), True),
    StructField("author_steamid", StringType(), True),
    StructField("author_num_games_owned", IntegerType(), True),
    StructField("author_num_reviews", IntegerType(), True),
    StructField("author_playtime_forever", IntegerType(), True),
    StructField("author_playtime_last_two_weeks", IntegerType(), True),
    StructField("author_playtime_at_review", IntegerType(), True),
    StructField("author_last_played", LongType(), True),
    StructField("language", StringType(), True),
    StructField("review", StringType(), True),
    StructField("timestamp_created", LongType(), True),
    StructField("timestamp_updated", LongType(), True),
    StructField("voted_up", IntegerType(), True),
    StructField("votes_up", IntegerType(), True),
    StructField("votes_funny", IntegerType(), True),
    StructField("weighted_vote_score", StringType(), True),
    StructField("comment_count", IntegerType(), True),
    StructField("steam_purchase", IntegerType(), True),
    StructField("received_for_free", IntegerType(), True),
    StructField("written_during_early_access", IntegerType(), True),
    StructField("hidden_in_steam_china", IntegerType(), True),
    StructField("steam_china_location", StringType(), True)
])

# Reading data from the landing folder
input_path = "gs://my-bigdata-project-bl/landing/all_reviews.csv"
sdf = spark.read.csv(input_path, schema=schema, header=True)

# Cleaning column names by removing spaces
sdf = sdf.select([col(column).alias(column.replace(" ", "_")) for column in sdf.columns])

# Dropping columns that are not needed
columns_to_drop = [
    'recommendationid',
    'appid',
    'author_steamid',
    'timestamp_updated',
    'hidden_in_steam_china',
    'steam_china_location',
    'votes_funny',
    'comment_count',
    'steam_purchase',
    'received_for_free',
    'written_during_early_access',
    'weighted_vote_score'
]

sdf = sdf.drop(*columns_to_drop)

# Filling in nulls or drop records with nulls
sdf = sdf.na.fill({
    'author_num_games_owned': 0,
    'author_num_reviews': 0,
    'author_playtime_forever': 0,
    'author_playtime_last_two_weeks': 0,
    'author_playtime_at_review': 0,
    'author_last_played': 0,
    'timestamp_created': 0,
    'votes_up': 0,
    'voted_up': 0
}).na.drop()

# Changing data types where ever necessary (ensure binary columns are integer type)
sdf = sdf.withColumn("voted_up", col("voted_up").cast(IntegerType()))

# Writing cleaned data to the cleaned folder as Parquet
output_path = "gs://my-bigdata-project-bl/cleaned/reviews_cleaned.parquet"
sdf.write.mode('overwrite').parquet(output_path)

In [ ]:
# Print the schema to confirm data types
sdf.printSchema()

root
 |-- game: string (nullable = true)
 |-- author_num_games_owned: integer (nullable = false)
 |-- author_num_reviews: integer (nullable = false)
 |-- author_playtime_forever: integer (nullable = false)
 |-- author_playtime_last_two_weeks: integer (nullable = false)
 |-- author_playtime_at_review: integer (nullable = false)
 |-- author_last_played: long (nullable = false)
 |-- language: string (nullable = true)
 |-- review: string (nullable = true)
 |-- timestamp_created: long (nullable = false)
 |-- voted_up: integer (nullable = false)
 |-- votes_up: integer (nullable = false)



In [ ]:
# Specify the correct path to the Parquet file
cleaned_parquet_path = "gs://my-bigdata-project-bl/cleaned/reviews_cleaned.parquet/"

# Load the Parquet file
reviews_df = spark.read.parquet(cleaned_parquet_path)

# Filter the DataFrame to include only English reviews (assuming 'language' column exists)
reviews_df = reviews_df.filter(reviews_df["language"] == "english")

# Drop the 'language' column
reviews_df = reviews_df.drop("language")

# Show the first few rows to verify the data was loaded correctly
reviews_df.show(5)


24/11/20 01:12:07 INFO SparkEnv: Registering MapOutputTracker
24/11/20 01:12:07 INFO SparkEnv: Registering BlockManagerMaster
24/11/20 01:12:07 INFO SparkEnv: Registering BlockManagerMasterHeartbeat
24/11/20 01:12:07 INFO SparkEnv: Registering OutputCommitCoordinator


+---------+----------------------+------------------+-----------------------+------------------------------+-------------------------+------------------+--------------------+-----------------+----------+----------+
|     game|author_num_games_owned|author_num_reviews|author_playtime_forever|author_playtime_last_two_weeks|author_playtime_at_review|author_last_played|              review|timestamp_created|  voted_up|  votes_up|
+---------+----------------------+------------------+-----------------------+------------------------------+-------------------------+------------------+--------------------+-----------------+----------+----------+
|Stellaris|                   318|                 5|                  34545|                             0|                    14785|        1684526083|Stelleris is one ...|       1532260250|         1|         0|
|Stellaris|                    67|                 1|                  24610|                             0|                    18695|      

In [ ]:
# Limit the Reviews to the top 300 games
from pyspark.sql.functions import count

# Count Game review frequency and sort
game_frequency_sdf = reviews_df.groupBy("game").agg(count("game").alias("frequency")).orderBy("frequency", ascending=False)

# Show result
game_frequency_sdf.show()

+--------------------+---------+
|                game|frequency|
+--------------------+---------+
|    Counter-Strike 2|  2129656|
|            Terraria|   607534|
|     Team Fortress 2|   599950|
|Tom Clancy's Rain...|   552871|
|  Grand Theft Auto V|   520192|
|                Rust|   507065|
|         Garry's Mod|   506496|
|            Among Us|   397216|
| PUBG: BATTLEGROUNDS|   395084|
|          ELDEN RING|   379213|
|        Apex Legends|   354954|
|        Phasmophobia|   333145|
|           Destiny 2|   327691|
|      Stardew Valley|   316343|
|ARK: Survival Evo...|   296160|
|       Rocket League|   290316|
|      Cyberpunk 2077|   280468|
|    Dead by Daylight|   268536|
|            PAYDAY 2|   254409|
|       Left 4 Dead 2|   247374|
+--------------------+---------+
only showing top 20 rows



In [ ]:
# Get the top 300 reviewed games
top_300_games_sdf = game_frequency_sdf.limit(300)

# Filter original DataFrame by doing an inner join with the top_300_games_sdf on 'game' column
top_300_reviews_sdf = reviews_df.join(top_300_games_sdf, "game")

# Show the result
top_300_reviews_sdf.show()

+---------+----------------------+------------------+-----------------------+------------------------------+-------------------------+------------------+--------------------+-----------------+----------+----------+---------+
|     game|author_num_games_owned|author_num_reviews|author_playtime_forever|author_playtime_last_two_weeks|author_playtime_at_review|author_last_played|              review|timestamp_created|  voted_up|  votes_up|frequency|
+---------+----------------------+------------------+-----------------------+------------------------------+-------------------------+------------------+--------------------+-----------------+----------+----------+---------+
|Stellaris|                   318|                 5|                  34545|                             0|                    14785|        1684526083|Stelleris is one ...|       1532260250|         1|         0|    84603|
|Stellaris|                    67|                 1|                  24610|                       

In [ ]:
# Save the resulting DataFrame to a Parquet file
output_path = "gs://my-bigdata-project-bl/cleaned/top_300_reviews.parquet"
top_300_reviews_sdf.write.mode("overwrite").parquet(output_path)

print(f"Filtered English reviews for top 300 games saved to {output_path}")

Filtered English reviews for top 300 games saved to gs://my-bigdata-project-bl/cleaned/top_300_reviews.parquet
